In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
kc=len(pd.read_csv('train_centers.csv'))

In [3]:
classes=['wave', 'inf', 'eight', 'circle', 'beat3', 'beat4']

In [4]:
train = {}
for cl in classes:
    train[cl]=pd.read_csv('train/'+cl+'.csv').to_numpy(int).flatten()

#### HMM

In [5]:
N=10
m=kc
B = np.zeros((N,kc)) #Nxm
B[:,:] = 1/m
A = np.zeros((N,N)) #NxN
A[:,:] = 1/N
prior= np.zeros((N)) #1xN
prior[0]=1

In [6]:
def alpha_calc(A,N,T,B,data,prior):
    
    alpha=np.zeros((N,T))
    c=np.zeros(T)
    
    alpha[:,0]=(B[:,data[0]]*prior)
    c[0]=1/(np.sum(alpha[:,0])+10**(-10))
    alpha[:,0]=alpha[:,0]*c[0]

    for t in range(1,T):
        alpha[:,t]=np.dot(alpha[:,t-1],A.T)*B[:,data[t]]
        c[t]=1/(np.sum(alpha[:,t])+10**(-10))
        alpha[:,t]=alpha[:,t]*c[t]
    return alpha,c

In [7]:
def beta_calc(N,data,prior,A,B,c):
    T=len(data)
    beta=np.zeros((N,T))
    beta[:,T-1]=c[0]
        
    for n in range(1,T):
        t = T-(n+1)
        for i in range (N):
            for j in range(N):
                beta[i,t] = beta[i,t] + A[i,j]*B[j,data[t+1]]*beta[j,t+1]
            beta[i,t] = c[t]*beta[i,t]
    return beta

In [8]:
def E_step(alpha,beta,T,N,A,B,data):
    gamma=np.zeros((N,T))
    epsilon=np.zeros((N,T,N))
    
    gamma[:,0]=(alpha[:,0]*beta[:,0])/(alpha[:,0].T@beta[:,0])
    
    for t in range (1,T):
        gamma[:,t]=(alpha[:,t]*beta[:,t])/(alpha[:,t].T@beta[:,t])
        h = T-(t+1)
        sums = 0
        for i in range (1,N):
            for j in range (1,N):  
                sums += alpha[i,t]*(A[i,j]*B[j,data[h]]*(beta[j,h+1]))
        epsilon[:,t,:] = (alpha[:,t]*(A*B[:,data[h+1]]@(beta[:,h+1])))/sums
    return gamma,epsilon

In [9]:
def M_step(data,gamma,N,T,epsilon,A,B,m):
    A_new=np.copy(A)
    B_new=np.copy(B)
    
    prior = gamma[:,0]/(np.sum(gamma[:,0]))
    
    for i in range (N):
        sums = np.sum(epsilon[i,:,:])
        for j in range (N):                    
            A_new[j,i] = np.sum(epsilon[i,:,j])/sums
            
    for i in range(N):
        den = 0 
        for t in range(T):
            den +=gamma[i][t]
        for j in range(m):
            num = 0
            for t in range(T):
                if (data[t] == j):
                    num +=gamma[i][t]
            B_new[i][j] = num/den
    
    return prior,A_new,B_new

In [10]:
def loglikelihood(c):    
    return -np.sum(np.log(c))

In [11]:
def BW_algo(A,N,B,m,data,prior,tol):
    prev=0
    T=len(data)
    while True:
        
        alpha,c=alpha_calc(A,N,T,B,data,prior)
        beta=beta_calc(N,data,prior,A,B,c)
        gamma,epsilon = E_step(alpha,beta,T,N,A,B,data)
        prior,A,B = M_step(data,gamma,N,T,epsilon,A,B,m)
        
        error=loglikelihood(c)
        
        print(error)
        if abs(error-prev)<=tol or np.isnan(error):
            print('converges')
            break
        else:
            prev=error
            
    return A,B,prior

In [12]:
As,Bs,Ps={},{},{}
for cl in classes:
    print('class',cl)
    As[cl],Bs[cl],Ps[cl]=BW_algo(A,N,B,m,train[cl],prior,1)

class wave
-56841.694268865955
-34971.49219291376
-34971.44059833545
converges
class inf
-48575.58965839629
-29857.843333685763
-29857.838556461706
converges
class eight
-55945.84100062291
-34159.07088502202
-34159.06197655674
converges
class circle
-41573.0684786799
-24709.265992736713
-24709.258310378667
converges
class beat3
-52542.380985900425
-36461.59858654433
-36461.59652705464
converges
class beat4
-54901.3308581736
-41622.918504762114
-41622.913904015964
converges


#### Testing on validation set

In [13]:
val = {}
for cl in classes:
    val[cl]=pd.read_csv('val/'+cl+'.csv').to_numpy(int).flatten()

In [14]:
def predict(As,Bs,N,data,Ps,classes):
    ll=np.array([])
    T=len(data)
    for cl in classes:
        a,c=alpha_calc(As[cl],N,T,Bs[cl],data,Ps[cl])
        ll=np.append(ll,-np.sum(np.log(c)))
        
    index=ll.argsort()[-3:][::-1]
    for i in range(3):
        ids=index[i]
        print(' ','Prediction',i,' ', 'Class:',classes[ids],'LL:',round(ll[ids],2))
    return

In [15]:
for cl in classes:
    print('Data:',cl)
    predict(As,Bs,N,val[cl],Ps,classes)

Data: wave
  Prediction 0   Class: wave LL: -1907.72
  Prediction 1   Class: beat3 LL: -5819.66
  Prediction 2   Class: beat4 LL: -5838.23
Data: inf
  Prediction 0   Class: inf LL: -1847.82
  Prediction 1   Class: beat3 LL: -12125.21
  Prediction 2   Class: beat4 LL: -12342.63
Data: eight
  Prediction 0   Class: eight LL: -1443.67
  Prediction 1   Class: inf LL: -8342.84
  Prediction 2   Class: beat3 LL: -9479.94
Data: circle
  Prediction 0   Class: circle LL: -1219.81
  Prediction 1   Class: beat4 LL: -6761.94
  Prediction 2   Class: beat3 LL: -7451.43
Data: beat3
  Prediction 0   Class: beat3 LL: -1717.44
  Prediction 1   Class: beat4 LL: -5893.17
  Prediction 2   Class: wave LL: -10237.05
Data: beat4
  Prediction 0   Class: beat4 LL: -2289.19
  Prediction 1   Class: beat3 LL: -10280.59
  Prediction 2   Class: wave LL: -12446.37


In [16]:
for cl in classes:
    pd.DataFrame(As[cl]).to_csv('models/'+'As_'+cl+'.csv',index=False)

In [17]:
for cl in classes:
    pd.DataFrame(Bs[cl]).to_csv('models/'+'Bs_'+cl+'.csv',index=False)

In [18]:
for cl in classes:
    pd.DataFrame(Ps[cl]).to_csv('models/'+'Ps_'+cl+'.csv',index=False)

In [19]:
with open('states_num.txt', 'w') as f:
    f.write('%d' % N)